In [ ]:
import requests
import json
import pandas as pd

In [ ]:


# # Google Apps Script Web App URL
# url = "https://script.google.com/macros/s/AKfycbwyP9x-mjl2wu8hYRP0Ctyep2BN4VDLmL9czfR-DoH2gi0JebPG-6M2v2SMuVWE0C_yDw/exec"
# url = 'http://127.0.0.1:5000/submit-stock'
# # JSON payload
# payload ={'State': 'Chhattisgarh', 'SS_Name': 'Agarwal Enterprises', 'DB_Name': 'Agarwal corpo.', 'Month': '2025-05', 'stocks': [{'Category': 'Bis', 'Item': 'Product A', 'Mrp': 100, 'Opening': '1238', 'Closing': '90'}, {'Category': 'Bis', 'Item': 'Product B', 'Mrp': 200, 'Opening': '0', 'Closing': '8'}, {'Category': 'Conf', 'Item': 'Product A', 'Mrp': 20, 'Opening': '78', 'Closing': '0'}, {'Category': 'Conf', 'Item': 'Product B', 'Mrp': 20, 'Opening': '8', 'Closing': '0'}]}

In [ ]:
# # Send POST request
# response = requests.post(url, json=payload)

# # Print response
# print("Status Code:", response.status_code)
# print("Response Text:", response.text)


## Data Csv to Json

In [ ]:
#df = pd.read_csv('Data/db.csv')
df1 = pd.read_excel('Data/sku_master.xlsx',sheet_name='sku_master')

In [ ]:
df1.to_json('Data/sku_master.json')

In [ ]:
df1.sort_values(by=['Category','Item_list','Mrp'],inplace=True)

In [ ]:
df2 = pd.read_json('Data/sku_master.json') 
df2

In [ ]:
df_ex = pd.read_excel('Data/db-excel.xlsx',sheet_name='selected')

In [ ]:
df_ex.columns

In [ ]:
df_ex1 = df_ex.rename(columns={
    'STATE NAME':'State',
    'SS NAME':'SS Name',
    'Dealer Name (As Per GST)':'DB Name',
    'Cross Checking':'Password'
})

In [ ]:
df_ex1 = df_ex1[['State', 'SS Name', 'DB Name', 'Password']]

In [ ]:
df_ex1= df_ex1.dropna()

In [ ]:
df_ex1['Password'] = df_ex1['Password'].astype(int)
df_ex1['State'] = df_ex1['State'].astype(str)
df_ex1['SS Name'] = df_ex1['SS Name'].astype(str)
df_ex1['DB Name'] = df_ex1['DB Name'].astype(str)

In [ ]:
df_ex1.sort_values(by=['State', 'SS Name', 'DB Name', 'Password'],inplace=True)

In [ ]:
# Create nested dictionary with password at DB level
nested = {}

for _, row in df_ex1.iterrows():
    state = row['State']
    ss = row['SS Name']
    db = row['DB Name'] # DB Name
    pwd = row['Password']

    state_dict = nested.setdefault(state, {})
    ss_dict = state_dict.setdefault(ss, {})
    db_dict = ss_dict.setdefault(db, {"password": pwd})


# Save
with open("Data/db.json", "w") as f:
    json.dump(nested, f, indent=4)